# Sentiment Analysis를 통한 강의평 기반 강의 추천

In [1]:
!pip install transformers
!pip install sklearn

     |████████████████████████████████| 2.3MB 30.7MB/s 
     |████████████████████████████████| 901kB 40.2MB/s 
     |████████████████████████████████| 3.3MB 38.9MB/s 


In [2]:
#주요 패키지들을 불러와줍니다.

import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [3]:
#구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [41]:
#model 불러오기
from sklearn.externals import joblib
model = joblib.load('gdrive/MyDrive/model.pkl')

In [42]:
def convert_data(sent):
  global final_test_sentence
  # tokenize and pad sentence
  sentence = ["[CLS] " + str(sent) + " [SEP]"]
  tokenized_texts = [tokenizer.tokenize(sentence[0])]
  labels = [1] #1로 고정, 상관 없음
  input_ids = [tokenizer.convert_tokens_to_ids(tokenized_texts[0])]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  #attention_masks
  attention_masks = []
  for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

  #파이토치 텐서로 변환
  test_inputs = torch.tensor(input_ids)
  test_labels = torch.tensor(labels)
  test_masks = torch.tensor(attention_masks)

  #배치 및 데이터로더 설정
  test_data = TensorDataset(test_inputs, test_masks, test_labels)
  test_sampler = RandomSampler(test_data)
  input_data = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)
  return input_data

In [43]:
def test_sentence(input_data):
  # 평가모드로 변경
  model.eval()

  # 변수 초기화
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # 데이터로더에서 배치만큼 반복하여 가져옴
  for step, batch in enumerate(input_data):
      # 배치를 GPU에 넣음
      batch = tuple(t.to(device) for t in batch)
      
      # 배치에서 데이터 추출
      b_input_ids, b_input_mask, b_labels = batch
      
      # 그래디언트 계산 안함
      with torch.no_grad():     
          # Forward 수행
          outputs = model(b_input_ids, 
                          token_type_ids=None, 
                          attention_mask=b_input_mask)
      
      ###################### get prediction! ######################
      logits = outputs[0]

      # CPU로 데이터 이동
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

      pred = np.argmax(logits)
      if pred == 1:
        result = "긍정"
      elif pred == 0:
        result = "부정"
      
      print(sentence, "\n위 문장은 " + result + "입니다.")

In [44]:
#GPU 체크 및 할당
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [45]:
#토크나이저, MAX_LEN, BATCH_SIZE 정의
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
MAX_LEN = 128
BATCH_SIZE = 32

In [46]:
# 직접 문장을 넣어서 확인할 수 있는 코드입니다.
sentence = "코로나 이후 비대면으로 강의가 전환되었는데도 불구하고 교수님께서 강의 준비를 꼼꼼히 해주시고 학생들도 신경 써 주시는 것 같아요."
test_sentence(convert_data(sentence))

코로나 이후 비대면으로 강의가 전환되었는데도 불구하고 교수님께서 강의 준비를 꼼꼼히 해주시고 학생들도 신경 써 주시는 것 같아요. 
위 문장은 긍정입니다.


In [47]:
# 직접 문장을 넣어서 확인할 수 있는 코드입니다.
sentence = "교수님 목소리가 너무 작고 강의력이 별로 안좋으신 것 같아요. 사실 수업에서 뭘 배우는지 잘 모르겠어요."
test_sentence(convert_data(sentence))

교수님 목소리가 너무 작고 강의력이 별로 안좋으신 것 같아요. 사실 수업에서 뭘 배우는지 잘 모르겠어요. 
위 문장은 부정입니다.


In [48]:
# 직접 문장을 넣어서 확인할 수 있는 코드입니다.
sentence = "힘들었지만 얻어가는 게 있었던 것 같아요."
test_sentence(convert_data(sentence))

힘들었지만 얻어가는 게 있었던 것 같아요. 
위 문장은 긍정입니다.


In [49]:
# 직접 문장을 넣어서 확인할 수 있는 코드입니다.
sentence = "교수님은 강의력 나쁘지 않고 괜찮으셨던 것 같은데 성적을 너무 낮게 주시는 것 같아요. 이 점 참고하셔서 들으셔야 할 것 같아요."
test_sentence(convert_data(sentence))

교수님은 강의력 나쁘지 않고 괜찮으셨던 것 같은데 성적을 너무 낮게 주시는 것 같아요. 이 점 참고하셔서 들으셔야 할 것 같아요. 
위 문장은 부정입니다.


In [50]:
# 직접 문장을 넣어서 확인할 수 있는 코드입니다.
sentence = "과제가 별로 없어서 수업 듣기 수월했습니다"
test_sentence(convert_data(sentence))

과제가 별로 없어서 수업 듣기 수월했습니다 
위 문장은 긍정입니다.


In [51]:
# 직접 문장을 넣어서 확인할 수 있는 코드입니다.
sentence = "전체적으로 무난했는데 교수님 성적 기준이 뭔지 모르겠어요"
test_sentence(convert_data(sentence))

전체적으로 무난했는데 교수님 성적 기준이 뭔지 모르겠어요 
위 문장은 부정입니다.


In [52]:
# 직접 문장을 넣어서 확인할 수 있는 코드입니다.
sentence = "일단 수업 듣는 내내 좋은 점은 딱 하나 있었던 것 같습니다. 할 게 별로 없었어요. 성적에 비해서 시키시는 것도 별로 없으시구요. \
그런데 저는 개인적으로 수업 들을 때 조금 힘들어도 얻는 게 많은 수업들을 좋아하는데 이 수업은 저랑 안 맞았어요. 얻어가는 게 하나도 없는 수업이라고 생각하시면 됩니다. \
과제같은 것들도 의미가 없게 느껴져서 수업 듣는 내내 너무 힘들었습니다."
test_sentence(convert_data(sentence))

일단 수업 듣는 내내 좋은 점은 딱 하나 있었던 것 같습니다. 할 게 별로 없었어요. 성적에 비해서 시키시는 것도 별로 없으시구요. 그런데 저는 개인적으로 수업 들을 때 조금 힘들어도 얻는 게 많은 수업들을 좋아하는데 이 수업은 저랑 안 맞았어요. 얻어가는 게 하나도 없는 수업이라고 생각하시면 됩니다. 과제같은 것들도 의미가 없게 느껴져서 수업 듣는 내내 너무 힘들었습니다. 
위 문장은 부정입니다.
